<a href="https://colab.research.google.com/github/shaheer1995/FuSENet/blob/master/FUSENet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone 'https://github.com/shaheer1995/FuSENet'

fatal: destination path 'FuSENet' already exists and is not an empty directory.


In [ ]:
!pip3 install keras==2.2.4

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os

init_notebook_mode(connected=True)
%matplotlib inline
import numpy as np
import random 
#import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.utils.np_utils import to_categorical
from sklearn.decomposition import PCA
from keras.optimizers import Adam, SGD, Adadelta, RMSprop, Nadam
import keras.callbacks as kcallbacks
from keras.regularizers import l2
import time
import collections
from sklearn import metrics, preprocessing
from keras import backend as K

Using TensorFlow backend.


In [ ]:
%mkdir Datasets
%cd Datasets

mkdir: cannot create directory ‘Datasets’: File exists
/content/Datasets


In [ ]:
!wget 'http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat'
!wget 'http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat'

--2020-12-29 07:24:15--  http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5953527 (5.7M)
Saving to: ‘Indian_pines_corrected.mat.1’

Indian_pines_correc 100%[===================>]   5.68M   591KB/s    in 12s     

2020-12-29 07:24:28 (493 KB/s) - ‘Indian_pines_corrected.mat.1’ saved [5953527/5953527]

--2020-12-29 07:24:28--  http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1125 (1.1K)
Saving to: ‘Indian_pines_gt.mat.1’

Indian_pines_gt.mat 100%[===================>]   1.10K  --.-KB/s    in 0s      

2020-12-29 07:24:28 (192 MB/s) - ‘Indian_pines_gt.mat.1’ saved [1125/1

In [ ]:
def loadData(name):
    data_path = '/content/Datasets'
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    elif name == 'KSC':
        data = sio.loadmat(os.path.join(data_path, 'KSC.mat'))['KSC']
        labels = sio.loadmat(os.path.join(data_path, 'KSC_gt.mat'))['KSC_gt']
    
    return data, labels

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
!cp /content/FuSENet/Utils/se_resnet8_max.py /content/
!cp /content/FuSENet/Utils/cbof.py /content/
%cd /content/

/content


In [ ]:
from google.colab import files
import se_resnet8_max

def res4_model_ss():
    model_res4 = se_resnet8_max.SE_ResNet8((1, w, w, K),64, mp[d])
    # Set the hyper-parameters
    # m set according to paper to 2 * number of bits
    #my_custom_loss = lambda x,y: hashing_loss(x, y, m=2*32, alpha=0.001)

    # Compile the model
    #opt = keras.optimizers.adam(lr=0.1)
    #model_res4.compile(loss=my_custom_loss, optimizer=opt,metrics=['accuracy'])
    RMS = RMSprop(lr=0.001)
    # Let's train the model using RMSprop
    model_res4.compile(loss='categorical_crossentropy', optimizer=RMS, metrics=['accuracy'])

    return model_res4





input shape: (None, 15, 15, 5, 32)
(?, 1, 1, 32)
input shape: (None, 13, 13, 1, 32)
(?, 1, 1, 32)
(None, 13, 13, 32)

call
output
(None, 13, 13, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 15, 15, 1 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 15, 15, 5, 32 256         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 15, 15, 5, 32 128         conv3d_1[0][0]                   
_______________________________________

In [ ]:
import os
data_path = '/content/FuSENet/' #Enter Desired output path here from utils
os.listdir(data_path)

['FUSENet.ipynb', 'README.md', 'LICENSE', 'FuSENet-Paper.pdf', 'Utils', '.git']

In [ ]:
!cp /content/FuSENet/Utils/utils3.rar /content/
%cd /content/

/content


In [ ]:
from google.colab import files
!unrar e -y utils3.rar
import zeroPadding
import normalization
import doPCA
import modelStatsRecord
import averageAccuracy
import ssrn_SS_IN


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from utils3.rar

Extracting  averageAccuracy.py                                             2%  OK 
Extracting  doPCA.py                                                       4%  OK 
Extracting  modelStatsRecord.py                                           11%  OK 
Extracting  New Text Document.txt                                         12%  OK 
Extracting  normalization.py                                              14%  OK 
Extracting  Rawdata_3D_ResNet.py                                          42%  OK 
Extracting  Real3D_conv.py                                                67%  OK 
Extracting  ssrn_SS_IN.py                                                 96%  OK 
Extracting  zeroPadding.py                                                99%  OK 
All OK


In [ ]:
## GLOBAL VARIABLES
dataset = ['IP']#,'IP','PU']#
test_ratio = [0.8]#
windowSize = [15]#
mp = {'IP':16,'PU':9,'SA':16}

In [ ]:
for d in dataset:
  for t in test_ratio:
    for w in windowSize:
      X , Y = loadData(d)
      K = 30 if d == 'IP' else 15
      X,pca = applyPCA(X,numComponents=K)
      X, Y = createImageCubes(X, Y, windowSize=w)
      X.shape, Y.shape
      sz = X.shape[0]//10
      Y = to_categorical(Y)
      X = X.reshape((X.shape[0],w,w,K,1))
      train_X, test_X, train_Y, test_Y = splitTrainTestSet(X, Y, t)
      val_X = test_X[:sz]
      test_X = test_X[sz:]
      val_Y = test_Y[:sz]
      test_Y = test_Y[sz:]
      KAPPA_RES_SS4 = []
      OA_RES_SS4 = []
      AA_RES_SS4 = []
      TRAINING_TIME_RES_SS4 = []
      TESTING_TIME_RES_SS4 = []
      ELEMENT_ACC_RES_SS4 = np.zeros((1, mp[d]))


      for index_iter in range(1):
          #try:
            print("# %d Iteration" % (index_iter + 1))

            # save the best validated model 

            best_weights_RES_path_ss4 = data_path+ '/' + str(
                index_iter + 1) + '_NEW_KERNEL32_B_32_hashing'+str(w)+'_'+str(int((1-t)*100))+'.hdf5'

            
            
            #Bilinear
            #
            #
            
            model_res4_SS_BN = res4_model_ss()

            earlyStopping6 = kcallbacks.EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='auto')
            saveBestModel6 = kcallbacks.ModelCheckpoint(best_weights_RES_path_ss4, monitor='val_loss', verbose=1,
                                                        save_best_only=True,
                                                        mode='auto')

            tic6 = time.clock()
            print(train_X.shape, test_X.shape)
            history_res4_SS_BN = model_res4_SS_BN.fit(
                train_X, train_Y,
                validation_data=(val_X, val_Y),
                batch_size=256,
                nb_epoch=50, shuffle=True, callbacks=[earlyStopping6,saveBestModel6])
            toc6 = time.clock()

            tic7 = time.clock()
            loss_and_metrics_res4_SS_BN = model_res4_SS_BN.evaluate(
                test_X, test_Y,
                batch_size=32)
            toc7 = time.clock()

            print('3D RES_SS4 without BN Training Time: ', toc6 - tic6)
            print('3D RES_SS4 without BN Test time:', toc7 - tic7)

            print('3D RES_SS4 without BN Test score:', loss_and_metrics_res4_SS_BN[0])
            print('3D RES_SS4 without BN Test accuracy:', loss_and_metrics_res4_SS_BN[1])

            print(history_res4_SS_BN.history.keys())

            pred_test_res4 = model_res4_SS_BN.predict(test_X).argmax(axis=1)
            collections.Counter(pred_test_res4)
            gt_test = test_Y
            overall_acc_res4 = metrics.accuracy_score(pred_test_res4, gt_test.argmax(axis=1))
            confusion_matrix_res4 = metrics.confusion_matrix(pred_test_res4, gt_test.argmax(axis=1))
            each_acc_res4, average_acc_res4 = averageAccuracy.AA_andEachClassAccuracy(confusion_matrix_res4)
            kappa = metrics.cohen_kappa_score(pred_test_res4, gt_test.argmax(axis=1))
            KAPPA_RES_SS4.append(kappa)
            OA_RES_SS4.append(overall_acc_res4)
            AA_RES_SS4.append(average_acc_res4)
            TRAINING_TIME_RES_SS4.append(toc6 - tic6)
            TESTING_TIME_RES_SS4.append(toc7 - tic7)
            ELEMENT_ACC_RES_SS4[index_iter, :] = each_acc_res4

            print("3D RESNET_SS4 without BN training finished.")
            print("# %d Iteration" % (index_iter + 1))

            modelStatsRecord.outputStats(KAPPA_RES_SS4, OA_RES_SS4, AA_RES_SS4, ELEMENT_ACC_RES_SS4,
                                     TRAINING_TIME_RES_SS4, TESTING_TIME_RES_SS4,
                                     history_res4_SS_BN, loss_and_metrics_res4_SS_BN, mp[d],
                                     data_path + '/IN_train_RESNET8_KERNEL64_20_B_32_Fuse_'+str(w)+'_'+str(int((1-t)*100))+'.txt',
                                     data_path + '/IN_train_RESNET8_KERNEL64_20_B_32_PU_element_9_Fuse_'+str(w)+'_'+str(int((1-t)*100))+'.txt')

# 1 Iteration
input shape: (None, 15, 15, 12, 32)
(?, 1, 1, 32)
input shape: (None, 13, 13, 1, 32)
(?, 1, 1, 32)
(None, 13, 13, 32)
call
output
(None, 13, 13, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 15, 15, 30, 1 0                                            
__________________________________________________________________________________________________
conv3d_9 (Conv3D)               (None, 15, 15, 12, 3 256         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 15, 15, 12, 3 128         conv3d_9[0][0]                   
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 15, 15

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.






Train on 2049 samples, validate on 1024 samples
Epoch 1/50





2049/2049 [==============================] - 49s 24ms/step - loss: 8.1152 - acc: 0.1005 - val_loss: 13.5282 - val_acc: 0.0918

Epoch 00001: val_loss improved from inf to 13.52818, saving model to /content/FuSENet//1_NEW_KERNEL32_B_32_hashing15_19.hdf5
Epoch 2/50
2049/2049 [==============================] - 2s 848us/step - loss: 8.5232 - acc: 0.1020 - val_loss: 13.3426 - val_acc: 0.0918

Epoch 00002: val_loss improved from 13.52818 to 13.34260, saving model to /content/FuSENet//1_NEW_KERNEL32_B_32_hashing15_19.hdf5
Epoch 3/50
2049/2049 [==============================] - 2s 829us/step - loss: 8.1555 - acc: 0.1122 - val_loss: 13.1523 - val_acc: 0.0918

Epoch 00003: val_loss improved from 13.34260 to 13.15227, saving model to /content/FuSENet//1_NEW_KERNEL32_B_32_hashing15_19.hdf5
Epoch 4/50
2049/2049 [==============================] - 2s 826us/step - loss: 7.2486 - acc: 0.1498 - val_loss: 10.7094 - val_acc: 0.0820

Epoch 0

/content/averageAccuracy.py:8: RuntimeWarning:

invalid value encountered in true_divide

